In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
# softmax 예시 (합은 1)
z = torch.FloatTensor([1,2,3])

hypothesis = torch.softmax(z, dim=0)
print(hypothesis)
print(hypothesis.sum())

tensor([0.0900, 0.2447, 0.6652])
tensor(1.)


In [37]:
# 연습1
z = torch.rand(3,5, requires_grad=True)
hypothesis = torch.softmax(z, dim = 1)
print(hypothesis)

tensor([[0.2807, 0.1525, 0.1651, 0.2446, 0.1571],
        [0.1342, 0.2656, 0.2184, 0.2535, 0.1284],
        [0.1959, 0.2806, 0.1352, 0.2408, 0.1476]], grad_fn=<SoftmaxBackward0>)


In [47]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([2, 3, 0])


In [51]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.]])

In [52]:
# 직접 cost
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

# 같은 표현
cost = (y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()
print(cost)

tensor(1.6015, grad_fn=<MeanBackward0>)


In [49]:
# 연습2 (nll loss로 손실함수 계산)

print(F.nll_loss(F.log_softmax(z, dim=1), y))

tensor(1.6015, grad_fn=<NllLossBackward0>)


In [50]:
# 연습3 (cross entropy 로 cost 계산)
print(F.cross_entropy(z, y))

tensor(1.6015, grad_fn=<NllLossBackward0>)


In [53]:
# 진짜 모델 구현
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [63]:
w = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([w,b], lr=0.1)

epochs = 1000
for epoch in range(1, epochs+1):
    hypothesis = F.softmax(x_train.matmul(w) + b, dim=1)
    z = x_train.matmul(w) + b

    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -F.log_softmax(z, dim=1)).sum(dim=1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch: {:4d}/{:4d}, Cost: {:.5f}'.format(epoch, epochs, cost.item()))

Epoch:  100/1000, Cost: 0.71427
Epoch:  200/1000, Cost: 0.63843
Epoch:  300/1000, Cost: 0.59119
Epoch:  400/1000, Cost: 0.55411
Epoch:  500/1000, Cost: 0.52203
Epoch:  600/1000, Cost: 0.49279
Epoch:  700/1000, Cost: 0.46523
Epoch:  800/1000, Cost: 0.43859
Epoch:  900/1000, Cost: 0.41227
Epoch: 1000/1000, Cost: 0.38580


In [62]:
w = torch.zeros((4,3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([w,b], lr=0.1)

epochs = 1000
for epoch in range(1, epochs+1):
    z = x_train.matmul(w) + b

    cost = F.cross_entropy(z, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch: {:4d}/{:4d}, Cost: {:.5f}'.format(epoch, epochs, cost.item()))

Epoch:  100/1000, Cost: 0.71427
Epoch:  200/1000, Cost: 0.63843
Epoch:  300/1000, Cost: 0.59119
Epoch:  400/1000, Cost: 0.55411
Epoch:  500/1000, Cost: 0.52203
Epoch:  600/1000, Cost: 0.49279
Epoch:  700/1000, Cost: 0.46523
Epoch:  800/1000, Cost: 0.43859
Epoch:  900/1000, Cost: 0.41227
Epoch: 1000/1000, Cost: 0.38580


In [67]:
print(w)
print(b)

tensor([[-3.6653, -0.1199,  3.7852],
        [-0.6077, -0.5192,  1.1270],
        [ 2.3670, -0.5431, -1.8239],
        [-0.5498,  1.3626, -0.8128]], requires_grad=True)
tensor([-5.3644e-08], requires_grad=True)


In [64]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3!

    def forward(self, x):
        return self.linear(x)

In [65]:
model = SoftmaxClassifierModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 2.075028
Epoch  100/1000 Cost: 0.711181
Epoch  200/1000 Cost: 0.626148
Epoch  300/1000 Cost: 0.568209
Epoch  400/1000 Cost: 0.517701
Epoch  500/1000 Cost: 0.470104
Epoch  600/1000 Cost: 0.423756
Epoch  700/1000 Cost: 0.377887
Epoch  800/1000 Cost: 0.332201
Epoch  900/1000 Cost: 0.287272
Epoch 1000/1000 Cost: 0.248987


In [71]:
print(model.parameters())

<generator object Module.parameters at 0x7b0378f54f20>
